In [19]:
pip install kafka-python


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pyspark


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import pyspark

In [22]:
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_2.13:{pyspark.__version__} pyspark-shell'
os.environ['SPARK_SUBMIT_OPTS'] = '-Djdk.security.auth.login.Config=ignore'


In [23]:
KAFKA_BROKER_URL = "localhost:9092"
KAFKA_TOPIC = "wikimedia_topic_1"

In [24]:
pip install requests


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install sseclient


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
import threading
import time
from kafka import KafkaConsumer
from kafka import KafkaProducer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, expr
from pyspark.sql.types import StructType, StringType, IntegerType
from pyspark.streaming import StreamingContext


In [27]:
# *** Note!!! **** At this point you need to have Kafka broker running. See Setup for Docker and Kafka.

producer = KafkaProducer(bootstrap_servers=KAFKA_BROKER_URL)

In [28]:
# *** Note!!! **** In order to read wikimedia changes, you need to obtain Access Token 
# See instructions in Readm about how to authenticate to Wikimedia page

In [29]:
import requests
from sseclient import SSEClient
URL = 'https://stream.wikimedia.org/v2/stream/recentchange'
headers = {
    "User-Agent": "[YOUR_APP_NAME_HERE]",
    "Authorization": "Bearer [YOUR_ACCESS_TOKEN_HERE]"
}
def relay():
    events = SSEClient(URL, headers=headers, timeout=30)
    for i in range(100):
        for event in events:
            if event.event == 'message' and event.data != None:
                message = event.data.encode("utf-8")
                producer.send(KAFKA_TOPIC, value=message)
                break
threading.Thread(target=relay).start()


In [30]:
# *** Note!!! **** Before continue to the next phase, make sure that you have a topic and events in it.
# See 'Lookup Kafka Topics' in the Readme.

In [31]:
spark = SparkSession.builder \
    .appName("PySpark-jupyter-streaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:{pyspark.__version__}") \
    .config("spark.sql.streaming.checkpointLocation", "./checkpoint") \
    .getOrCreate()

In [32]:
kafka_df = spark.readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BROKER_URL) \
  .option("subscribe", KAFKA_TOPIC) \
  .option("startingOffsets", "earliest") \
  .load()

In [33]:
schema = StructType() \
    .add("id", IntegerType()) \
    .add("type", StringType()) \
    .add("comment", StringType()) \
    .add("user", StringType()) \
    .add("title", StringType()) \
    .add("server_name", StringType())

# Transform data to dataframe of json format
parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

In [34]:
parsed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start() 

26/01/15 19:48:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [35]:
parsed_df.createOrReplaceTempView("parsed_df")

spark.sql("select user, count(*) as count from parsed_df group by user") \
.writeStream \
.outputMode("complete") \
.format("console") \
.start() 

26/01/15 19:48:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [36]:
spark.sql("select type, count(*) as count from parsed_df group by type") \
.writeStream \
.outputMode("complete") \
.format("console") \
.start() 

26/01/15 19:48:08 WARN MicroBatchExecution: Disabling AQE since AQE is not supported in stateful workloads.
26/01/15 19:48:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


26/01/15 19:48:08 WARN MicroBatchExecution: Disabling AQE since AQE is not supported in stateful workloads.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+----------+----------+--------------------+------------------+--------------------+--------------------+
|        id|      type|             comment|              user|               title|         server_name|
+----------+----------+--------------------+------------------+--------------------+--------------------+
|      NULL|      NULL|                NULL|              NULL|                NULL|                NULL|
| 134121921|       log|Bot: Mass deletin...|         Ladsgroup|Thảo luận Thành v...|    vi.wikipedia.org|
|      NULL|       log|[Точність: перегл...|        Abinitio25|        Жорж Мустакі|    uk.wikipedia.org|
|      NULL|categorize|[[:File:20250412a...|PantheraLeo1359531|Category:Images f...|commons.wikimedia...|
|      NULL|categorize|[[:File:20250412a...|PantheraLeo1359531|Category:Files fr...|commons.wikimedia...|
|1986297315|      edit|The wrong William...|     ~2026-

-------------------------------------------
Batch: 0
-------------------------------------------


[Stage 9:==>            (27 + 17) / 200][Stage 10:>                 (0 + 0) / 1]

+------------------+-----+
|              user|count|
+------------------+-----+
|         MacTire02|   14|
|        Rathfelder|   16|
|            Geagea|    1|
|        CheWikibot|   11|
|       ListeriaBot|    1|
|        Bot-Jagwar|    1|
|     ~2026-31829-6|    1|
|              NULL|    1|
|        Liberliger|    1|
|           Tegebot|    2|
|     ~2026-32176-3|    1|
|        Abinitio25|    1|
|InternetArchiveBot|    1|
|          うにょす|    1|
|           Bigmorr|    1|
|            AVSBot|    1|
|     ~2026-30034-0|    4|
|          DPLA bot|    1|
|      Caddyshack01|    3|
|      GdyniaPL2025|    7|
+------------------+-----+
only showing top 20 rows


-------------------------------------------
Batch: 0
-------------------------------------------
+----------+-----+
|      type|count|
+----------+-----+
|       new|    4|
|       log|    8|
|      NULL|    1|
|      edit|   28|
|categorize|   59|
+----------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+----------+----------+--------------------+------------------+--------------------+--------------------+
|        id|      type|             comment|              user|               title|         server_name|
+----------+----------+--------------------+------------------+--------------------+--------------------+
|      NULL|      NULL|                NULL|              NULL|                NULL|                NULL|
|      NULL|categorize|[[:File:The Profe...|        Rathfelder|Category:The Iris...|commons.wikimedia...|
|1986299680|      edit|                    |   Johnpacklambert|Category:Deaths f...|    en.wikipedia.org

[Stage 12:(18 + 16) / 200][Stage 13:>   (0 + 0) / 1][Stage 14:>   (0 + 0) / 1]1]